# Cleaning - Overwatch League 2018 Player Statistics
---
Author: Max Tollefsen

Data Source: https://overwatchleague.com/en-us/statslab

### Overwatch and the Overwatch League

Overwatch is a 6 vs 6 first-person class-based shooter video game with MOBA elements. The Overwatch League (OWL) is to Overwatch what the NFL is to American football. Every year, or season, there are (generally) four stages of matches and then the playoffs at the end of the season.

### The Data

The dataset is one I made by taking all the datasets found on the [Stats Lab page of the Overwatch League website](#https://overwatchleague.com/en-us/statslab) and combining them in SQLite. It includes all player statistics from OWL matches in the 2018 season, OWL's inaugural year.

It has 10 variables (columns):
- `start_time` - The date and time that the match began.
- `match_id` - Unique ID of the match.
- `stage` - What stage in the Overwatch League season it is. 
- `map_type` - The game mode the match was played on (this can be Assault, Control, Hybrid, or Payload).
- `map_name` - Name of the map the match was played on.
- `player` - Name of the player.
- `team` - Name of the team the player was on.
- `stat_name` - Name of the statistic that was measured.
- `hero` - Name of the hero that was played.
- `stat_amount` - The measure of the stat_name.

### Importing & Cleaning the Data

First, I import some modules as well as my dataset from my SQLite database.

In [1]:
import pandas as pd
import sqlite3

#   Imports 2018 matches from SQL
database = r'C:\Users\Max\Google Drive\Data\SQL Database\Scrying Pool.db'
conn = sqlite3.connect(database)

OWL_2018 = pd.read_sql('select * from OWL_2018_Raw', con=conn)
conn.close()

#   Gives an overview of the data
print(OWL_2018.info())
print()
OWL_2018.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1124418 entries, 0 to 1124417
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   start_time   1124418 non-null  object
 1   match_id     1124418 non-null  object
 2   stage        1124418 non-null  object
 3   map_type     1124418 non-null  object
 4   map_name     1124418 non-null  object
 5   player       1124418 non-null  object
 6   team         1124418 non-null  object
 7   stat_name    1124418 non-null  object
 8   hero         1124418 non-null  object
 9   stat_amount  1124418 non-null  object
dtypes: object(10)
memory usage: 85.8+ MB
None



,start_time,match_id,stage,map_type,map_name,player,team,stat_name,hero,stat_amount
0,01-11-18 00:12,10223,Overwatch League - Stage 1,PAYLOAD,Dorado,Agilities,Los Angeles Valiant,All Damage Done,All Heroes,18079.20692
1,01-11-18 00:12,10223,Overwatch League - Stage 1,PAYLOAD,Dorado,Agilities,Los Angeles Valiant,Assists,All Heroes,17
2,01-11-18 00:12,10223,Overwatch League - Stage 1,PAYLOAD,Dorado,Agilities,Los Angeles Valiant,Barrier Damage Done,All Heroes,2893.659185
3,01-11-18 00:12,10223,Overwatch League - Stage 1,PAYLOAD,Dorado,Agilities,Los Angeles Valiant,Damage - Quick Melee,All Heroes,443.20401
4,01-11-18 00:12,10223,Overwatch League - Stage 1,PAYLOAD,Dorado,Agilities,Los Angeles Valiant,Deaths,All Heroes,11


There are no null values in our table which is good. Now I want to make some changes to the data so it is easier to handle for analysis.

To save memory `stage`, `map_type`, `map_name`, `player`, `team`, `stat_name`, and `hero` can be converted to the categorical dtype. Before doing this Let's get an idea of how many unique values are in each of the columns.

In [2]:
OWL_2018.nunique()

start_time       1109
match_id          262
stage               9
map_type            4
map_name           17
player            129
team               12
stat_name         284
hero               30
stat_amount    455086
dtype: int64

`stage` is interesting because there are only five stages in a season (stages 1 through 4 and the playoffs). So why does it have nine unique values? I'll come back to this after handling the other columns.

`map_type`, `map_name`, `team`, and `hero` make sense to make categorical data. There are a low number of values each variable can take on and it’s reasonable to assume the number of values they can take on in the future won't increase by a lot in a short amount of time (e.g. I don't expect a release of 12 heroes over three months, or for 35 new teams to join the league in between seasons).

`player` and `stat_name` have enough unique values that I'm hesitant to make them categorical data. Players can fluctuate some as old players leave and new talent joins the league. `stat_name` won't fluctuate as much since it's almost exclusively tied to the number of heroes in the game (a new hero can mean around 10 new stats being added that are unique to that hero) but at 284 unique values I'm not sure how much memory is saved so I'll do a side-by-side comparison of the two dtypes.

In [3]:
#    Converts the list of columns to categorical data
for col in ['map_type', 'map_name', 'team', 'hero']:
    OWL_2018[col] = OWL_2018[col].astype('category')

#   Creates a series for stat_name where it is the categorical dtype
stat_name_cat = OWL_2018.stat_name.astype('category')

#   Compares the memory usage of stat_name if it were an object or category dtype in megabytes
print( 'object type:   ' + str( OWL_2018.stat_name.memory_usage(deep=True)/1000000 )  + ' MB'  )
print( 'category type:  ' + str( stat_name_cat.memory_usage(deep=True)/1000000 ) + ' MB')

object type:   82.833791 MB
category type:  2.278881 MB


This is a ~97% reduction in memory size! The reduction seems significant enough to convert `player` and `stat_name` to categorical data as well.

In [4]:
#    Converting player and stat_name to categorical data
for col in ['player', 'stat_name']:
    OWL_2018[col] = OWL_2018[col].astype('category')

Now to revisit `stage`.

In [5]:
#   Displays all unique values in stage
OWL_2018.stage.unique()

array(['Overwatch League - Stage 1',
       'Overwatch League - Stage 1 - Title Matches',
       'Overwatch League - Stage 2',
       'Overwatch League - Stage 2 Title Matches',
       'Overwatch League - Stage 3',
       'Overwatch League - Stage 3 Title Matches',
       'Overwatch League - Stage 4',
       'Overwatch League - Stage 4 Title Matches',
       'Overwatch League Inaugural Season Championship'], dtype=object)

It seems `stage` indicates the stage the match took place in as well as if it was a title match. That is why it has nine unique values despite there only being five stages. I want to clean up the data so it only indicates what stage the match was in but I also want to keep the information on whether it was a title match or not so I create and insert a dummy column called `title_match` that shows True if the match was a title match.

In [6]:
#   Creating and inserting the title_match variable which indicates if the match is a title match or not
titleMatchBoolean = OWL_2018.stage.str.contains('Title Matches')
OWL_2018.insert(3, 'title_match', titleMatchBoolean)

Before making `stage` a category variable, I remove the portions of the data that don't provide any useful information and I add the year that the match took place. This is redundant information to add since all the matches in my dataset took place in 2018, but by adding the year I can distinguish stages from different seasons, which will be helpful if I were to ever add matches from a different season to this dataset.

In [7]:
#  Removes unnecessary portions of the string in stage and adds the year the match took place
OWL_2018.stage = OWL_2018.stage.str.replace('Overwatch League -', '2018')
OWL_2018.stage = OWL_2018.stage.str.replace('Overwatch League Inaugural Season Championship', '2018 Playoffs')
OWL_2018.stage = OWL_2018.stage.str.replace(' Title Matches', '')
OWL_2018.stage = OWL_2018.stage.str.replace(' -', '')

#   Converts stage to category dtype
OWL_2018.stage = OWL_2018.stage.astype('category')

OWL_2018.stage.unique()

['2018 Stage 1', '2018 Stage 2', '2018 Stage 3', '2018 Stage 4', '2018 Playoffs']
Categories (5, object): ['2018 Stage 1', '2018 Stage 2', '2018 Stage 3', '2018 Stage 4', '2018 Playoffs']

Now I look to `start_time` which I want to convert to the datetime dtype. However, the dates in `start_time` are in two different formats. `01-11-18 00:12` and `1/27/2018 1:41` are examples of each format. Before I change the dtype I edit the column to have one consistent format throughout.

In [8]:
#   Establishing three rows that have the first format and three rows that have the second format
#   so I can check on them as I am editing
print(OWL_2018.start_time.iloc[0:3])
print()
print(OWL_2018.start_time.iloc[128000:128003])

0    01-11-18 00:12
1    01-11-18 00:12
2    01-11-18 00:12
Name: start_time, dtype: object

128000    1/27/2018 1:41
128001    1/27/2018 1:41
128002    1/27/2018 1:41
Name: start_time, dtype: object


I start by replacing all instances of slashes '/' with dashes '-'. Then I create a function, `formatFix`, that ensures the day, month, and year fields of every date is two digits. Specifically, the function:

1. Adds a leading '0' to the hour field where it is only one digit
2. Adds a leading '0' to the month field where it is only one digit
3. Removes the century portion of the year field from any dates that have it

I apply this function to `start_time` before converting it to the datetime dtype.

In [9]:
OWL_2018.start_time = OWL_2018.start_time.str.replace('/', '-')

def formatFix(date: str) -> str:
    #   makes sure all hours are double digit
    if date[-5] == ' ':
        date = date[:-4] + '0' + date[-4:]
       
    #   makes sure all months are double digit
    if date[0] == '1' and date[1] == '-':
        date = '0' + date
        
    #   removes the century from the year
    if date[-10:-8] == '20':
        date = date[:-10] + date[-8:]

    return date

#   Applying the above function to all dates
OWL_2018.start_time = OWL_2018.start_time.apply(formatFix)

#   Converting column to datetime
OWL_2018.start_time = pd.to_datetime(OWL_2018.start_time, format='%m-%d-%y %H:%M')

#   Checking back on the rows that I earmarked earlier
print(OWL_2018.start_time.iloc[0:3])
print()
print(OWL_2018.start_time.iloc[128000:128003])

0   2018-01-11 00:12:00
1   2018-01-11 00:12:00
2   2018-01-11 00:12:00
Name: start_time, dtype: datetime64[ns]

128000   2018-01-27 01:41:00
128001   2018-01-27 01:41:00
128002   2018-01-27 01:41:00
Name: start_time, dtype: datetime64[ns]


The last changes I make to this dataset; I apply the capitalize string method (changes strings so the first character is upper case, all other lower case) to `map_type`, convert `match_id` to integer, and convert `stat_amount` to floating point number. I also upload the now clean dataset into my SQL database.

In [10]:
#   This is an example of the different approach you have to take in order
#   to maintain a column's category dtype. The first line is what
#   you would normally type to apply a string method to a column.
#   The second line is how to apply a string method to a column while
#   maintaining its category dtype

#OWL_2018.map_type = OWL_2018.map_type.str.capitalize()
OWL_2018.map_type = OWL_2018.map_type.cat.rename_categories(str.capitalize)

OWL_2018.match_id = OWL_2018.match_id.astype(int)
OWL_2018.stat_amount = OWL_2018.stat_amount.astype(float)

#   Uploads the current dataset to my SQL database,
#   commented out as I've already uploaded a copy to my SQL database
#conn = sqlite3.connect(database)
#OWL_2018.to_sql(name="OWL_2018", con=conn)
#conn.close()

#   A look at the dataset post cleanup
print(OWL_2018.info())
print()
OWL_2018.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1124418 entries, 0 to 1124417
Data columns (total 11 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   start_time   1124418 non-null  datetime64[ns]
 1   match_id     1124418 non-null  int32         
 2   stage        1124418 non-null  category      
 3   title_match  1124418 non-null  bool          
 4   map_type     1124418 non-null  category      
 5   map_name     1124418 non-null  category      
 6   player       1124418 non-null  category      
 7   team         1124418 non-null  category      
 8   stat_name    1124418 non-null  category      
 9   hero         1124418 non-null  category      
 10  stat_amount  1124418 non-null  float64       
dtypes: bool(1), category(7), datetime64[ns](1), float64(1), int32(1)
memory usage: 32.2 MB
None



,start_time,match_id,stage,title_match,map_type,map_name,player,team,stat_name,hero,stat_amount
0,2018-01-11 00:12:00,10223,2018 Stage 1,False,Payload,Dorado,Agilities,Los Angeles Valiant,All Damage Done,All Heroes,18079.206920
1,2018-01-11 00:12:00,10223,2018 Stage 1,False,Payload,Dorado,Agilities,Los Angeles Valiant,Assists,All Heroes,17.000000
2,2018-01-11 00:12:00,10223,2018 Stage 1,False,Payload,Dorado,Agilities,Los Angeles Valiant,Barrier Damage Done,All Heroes,2893.659185
3,2018-01-11 00:12:00,10223,2018 Stage 1,False,Payload,Dorado,Agilities,Los Angeles Valiant,Damage - Quick Melee,All Heroes,443.204010
4,2018-01-11 00:12:00,10223,2018 Stage 1,False,Payload,Dorado,Agilities,Los Angeles Valiant,Deaths,All Heroes,11.000000
